In [1]:
from Matcher.Utils import separate_groups, filter_applicant, connect_db_and_get_table
from Matcher.ScoreCaculator import CompatibilityScoreCaculator, ScoreMatrixCalculator
from Matcher.Hungarian import prepareMatrix, hungarian

import numpy as np

MYSQL_DB_PATH = "./db.sqlite3"
TABLE_NAME = "applicant"

df = connect_db_and_get_table(MYSQL_DB_PATH, TABLE_NAME)
df = filter_applicant(df)

hetro_F, hetro_M, homo_F, homo_M = separate_groups(df)
hetro_F = hetro_F.sample(frac=1).reset_index(drop=True)
hetro_M = hetro_M.sample(frac=1).reset_index(drop=True)
homo_F = homo_F.sample(frac=1).reset_index(drop=True)
homo_M = homo_M.sample(frac=1).reset_index(drop=True)

compatibility_score_caculator = CompatibilityScoreCaculator(
    
)
score_matrix_calculator = ScoreMatrixCalculator(compatibility_score_caculator)

score_matrix, score_F_to_M, score_M_to_F = score_matrix_calculator.get_hetrogenous_group_score_matrix(hetro_F, hetro_M)
score_matrix_transpose = score_matrix.T

pairing_result = hungarian(prepareMatrix(score_matrix))


num_hetro_f = len(hetro_F)
num_hetro_m = len(hetro_M)

In [ ]:
paired = []
failed_m = []
failed_f = []
for f, m in pairing_result:
    if f >= num_hetro_f:
        wxid = hetro_M.iloc[m]['wxid']
        failed_m.append(wxid)
        continue
    if m >= num_hetro_m:
        wxid = hetro_F.iloc[f]['wxid']
        failed_f.append(wxid)
        continue

    if score_matrix[f, m] == 0:
        wxid = hetro_M.iloc[m]['wxid']
        failed_m.append(wxid)
        wxid = hetro_F.iloc[f]['wxid']
        failed_f.append(wxid)
        continue
    score = score_matrix[f, m]
    paired.append((f, m, score))
failed_f.sort()
failed_m.sort()
paired.sort(key=lambda x: x[2], reverse=True)
from pprint import pprint
print(f"totoal females: {num_hetro_f}, males: {num_hetro_m}\n")
print(f"failed to pair female: {failed_f}")
print(f"failed num: {len(failed_f)}, failing rate: {len(failed_f) / num_hetro_f}\n")
print(f"failed to pair male: {failed_m}")
print(f"failed num: {len(failed_m)}, failing rate: {len(failed_m) / num_hetro_m}\n")

for f, m, score in paired:
    print(f"female: {hetro_F.iloc[f]['wxid']} and male: {hetro_M.iloc[m]['wxid']} paired with score: {score}, f to m: {score_F_to_M[f, m]}, m to f: {score_M_to_F[m, f]}")
    # print(f"Female: {f} and Male: {m} paired with score: {score_matrix[f, m]}, f to m: {score_F_to_M[f, m]}, m to f: {score_M_to_F[m, f]}")


In [ ]:
from pprint import pprint

debug_fields = ["mbti_ei", "mbti_sn", "mbti_tf", "mbti_jp", "preferred_mbti_ei", "preferred_mbti_sn", "preferred_mbti_tf", "preferred_mbti_jp", "grade", "hobby1", "hobby2", "hobby3", "travel_destination", "superpower", "use_of_money", "family", "lifestyle", "comment"]

for i in range(20):
    f_wxid = hetro_F.iloc[paired[i][0]]['wxid']
    m_wxid = hetro_M.iloc[paired[i][1]]['wxid']
    f_debug = hetro_F[hetro_F['wxid'] == f_wxid][debug_fields].to_dict(orient='records')[0]
    m_debug = hetro_M[hetro_M['wxid'] == m_wxid][debug_fields].to_dict(orient='records')[0]
    f_debug['mbti'] = [f_debug.pop('mbti_ei'), f_debug.pop('mbti_sn'), f_debug.pop('mbti_tf'), f_debug.pop('mbti_jp')]
    m_debug['mbti'] = [m_debug.pop('mbti_ei'), m_debug.pop('mbti_sn'), m_debug.pop('mbti_tf'), m_debug.pop('mbti_jp')]
    f_debug['preferred_mbti'] = [f_debug.pop('preferred_mbti_ei'), f_debug.pop('preferred_mbti_sn'), f_debug.pop('preferred_mbti_tf'), f_debug.pop('preferred_mbti_jp')]
    m_debug['preferred_mbti'] = [m_debug.pop('preferred_mbti_ei'), m_debug.pop('preferred_mbti_sn'), m_debug.pop('preferred_mbti_tf'), m_debug.pop('preferred_mbti_jp')]
    f_debug['mbti_info'] = {
        "mbti": f_debug.pop('mbti'),
        "preferred_mbti": f_debug.pop('preferred_mbti')
    }
    m_debug['mbti_info'] = {
        "mbti": m_debug.pop('mbti'),
        "preferred_mbti": m_debug.pop('preferred_mbti')
    }
    f_debug['hobbies'] = [f_debug.pop('hobby1'), f_debug.pop('hobby2'), f_debug.pop('hobby3')]
    m_debug['hobbies'] = [m_debug.pop('hobby1'), m_debug.pop('hobby2'), m_debug.pop('hobby3')]
    f_debug['questions'] = {
        "travel_destination": f_debug.pop('travel_destination'),
        "superpower": f_debug.pop('superpower'),
        "use_of_money": f_debug.pop('use_of_money'),
        "family": f_debug.pop('family'),
        "lifestyle": f_debug.pop('lifestyle')
    }
    m_debug['questions'] = {
        "travel_destination": m_debug.pop('travel_destination'),
        "superpower": m_debug.pop('superpower'),
        "use_of_money": m_debug.pop('use_of_money'),
        "family": m_debug.pop('family'),
        "lifestyle": m_debug.pop('lifestyle')
    }
    print(f"Pair {i+1}:")
    print(f"She likes him by {score_F_to_M[paired[i][0], paired[i][1]]}")
    pprint(f_debug)
    print(f"\nHe likes her by {score_M_to_F[paired[i][1], paired[i][0]]}")
    pprint(m_debug)
    print("\n")

In [ ]:
female_wxid = 'FILLMEIN'
pair_record = None

for f, m, score in paired:
    if hetro_F.iloc[f]['wxid'] == female_wxid:
        pair_record = (f, m, score)
        print(f"Female index: {f}, Male index: {m}")
        break

if pair_record:
    f, m, score = pair_record
    print(f"Female: {hetro_F.iloc[f]['wxid']} paired with Male: {hetro_M.iloc[m]['wxid']} with score: {score}")
    m_wxid = hetro_M.iloc[m]['wxid']
    m_debug = hetro_M[hetro_M['wxid'] == m_wxid][debug_fields].to_dict(orient='records')[0]
    m_debug['mbti'] = [m_debug.pop('mbti_ei'), m_debug.pop('mbti_sn'), m_debug.pop('mbti_tf'), m_debug.pop('mbti_jp')]
    m_debug['preferred_mbti'] = [m_debug.pop('preferred_mbti_ei'), m_debug.pop('preferred_mbti_sn'), m_debug.pop('preferred_mbti_tf'), m_debug.pop('preferred_mbti_jp')]
    m_debug['mbti_info'] = {
        "mbti": m_debug.pop('mbti'),
        "preferred_mbti": m_debug.pop('preferred_mbti')
    }
    m_debug['hobbies'] = [m_debug.pop('hobby1'), m_debug.pop('hobby2'), m_debug.pop('hobby3')]
    m_debug['questions'] = {
        "travel_destination": m_debug.pop('travel_destination'),
        "superpower": m_debug.pop('superpower'),
        "use_of_money": m_debug.pop('use_of_money'),
        "family": m_debug.pop('family'),
        "lifestyle": m_debug.pop('lifestyle')
    }
    pprint(m_debug)
else:
    print(f"No pair found for female wxid: {female_wxid}")

In [ ]:
male_wxid = 'FILLMEIN'
pair_record = None

for f, m, score in paired:
    if hetro_M.iloc[m]['wxid'] == male_wxid:
        pair_record = (f, m, score)
        print(f"Male index: {m}, Female index: {f}")
        break

if pair_record:
    f, m, score = pair_record
    print(f"Male: {hetro_M.iloc[m]['wxid']} paired with Female: {hetro_F.iloc[f]['wxid']} with score: {score}")
    f_wxid = hetro_F.iloc[f]['wxid']
    f_debug = hetro_F[hetro_F['wxid'] == f_wxid][debug_fields].to_dict(orient='records')[0]
    f_debug['mbti'] = [f_debug.pop('mbti_ei'), f_debug.pop('mbti_sn'), f_debug.pop('mbti_tf'), f_debug.pop('mbti_jp')]
    f_debug['preferred_mbti'] = [f_debug.pop('preferred_mbti_ei'), f_debug.pop('preferred_mbti_sn'), f_debug.pop('preferred_mbti_tf'), f_debug.pop('preferred_mbti_jp')]
    f_debug['mbti_info'] = {
        "mbti": f_debug.pop('mbti'),
        "preferred_mbti": f_debug.pop('preferred_mbti')
    }
    f_debug['hobbies'] = [f_debug.pop('hobby1'), f_debug.pop('hobby2'), f_debug.pop('hobby3')]
    f_debug['questions'] = {
        "travel_destination": f_debug.pop('travel_destination'),
        "superpower": f_debug.pop('superpower'),
        "use_of_money": f_debug.pop('use_of_money'),
        "family": f_debug.pop('family'),
        "lifestyle": f_debug.pop('lifestyle')
    }
    pprint(f_debug)
else:
    print(f"No pair found for male wxid: {male_wxid}")